<a href="https://colab.research.google.com/github/Fuenfgeld/ATeamDatenmanagementUndArchivierung/blob/main/IMECOS_spielwiese_felix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# import libraries
import sqlite3 as sq
from sqlite3 import Error
import pandas as pd
import numpy as np
import requests


In [3]:
#mount drive to access data
from google.colab import drive 
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [4]:
# SQLite connection
def create_connection_file():
    """ create a database connection to a database that resides
        in a file(db_file/imecos.db)
    """
    conn = None;
    try:
       #Establishing the connection
        conn = sq.connect('/content/gdrive/Shareddrives/IMECOS/imecos.db')
        return conn
        print(sqlite3.version)
    except Error as e:
        print("Error while connecting to sqlite", e)
    # finally:
    #     if conn:
    #         conn.close() 
    # Creating a temporary empty database in the memory.
conn = create_connection_file()
# Creating a cursor object using the cursor() method
cur = conn.cursor()
print("Successfully Connected to SQLite")

Successfully Connected to SQLite


In [ ]:
# material_path = "/content/drive/Shareddrives/05_BIDS_Datenmanagement & Archivierung im Umfeld der Forschung/Material"

In [ ]:
# !ls "/content/drive/Shareddrives/05_BIDS_Datenmanagement & Archivierung im Umfeld der Forschung/Material/source_dbs"

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [5]:
df = pd.read_csv('/content/gdrive/Shareddrives/IMECOS/csv_data.zip (Unzipped Files)/csv_data/allergy/careplans.csv') 
#print(df.head)
print(df.loc[0])

Id                   b066696b-4a2f-ec29-b018-bcf7464d020c
START                                          1943-02-15
STOP                                                  NaN
PATIENT              3575b903-dbd0-1d55-6146-9e8aa4ed52a5
ENCOUNTER            fd59905a-4ec8-62a6-d45d-d2a458d60111
CODE                                            384758001
DESCRIPTION           Self-care interventions (procedure)
REASONCODE                                            NaN
REASONDESCRIPTION                                     NaN
Name: 0, dtype: object


In [ ]:
cursor = conn.cursor()
#cursor.execute('drop table people_info')
#cursor.execute('CREATE TABLE people_info (Name nvarchar(50), Country nvarchar(50), Age int)')
cursor.execute('INSERT INTO people_info VALUES (12, 13, 14)')
cursor.execute('select * from people_info')

In [38]:
sql_create_source_data = requests.get('https://raw.githubusercontent.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/main/create_statements_source_db.sql').text
cursor = conn.cursor()
cursor.executescript(sql_create_source_data)
conn.commit()


In [39]:
cursor.execute("SELECT name FROM sqlite_master WHERE type = 'table'")
print(cursor.fetchall())

[('tb_patients',), ('tb_organizations',), ('tb_providers',), ('tb_payers',), ('tb_payer_transitions',), ('tb_encounters',), ('tb_careplans',), ('tb_conditions',), ('tb_devices',), ('tb_disease',), ('tb_imaging_studies',), ('tb_immunizations',), ('tb_observations',), ('tb_procedures',), ('tb_supplies',), ('tb_medications',)]


In [24]:
df

,Id,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,REASONCODE,REASONDESCRIPTION
0,b066696b-4a2f-ec29-b018-bcf7464d020c,1943-02-15,NaN,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,fd59905a-4ec8-62a6-d45d-d2a458d60111,384758001,Self-care interventions (procedure),NaN,NaN
1,8a34101d-6503-4181-6d2d-d316e5f71de0,2010-01-20,NaN,aff157cc-b6d3-412b-ccbe-bfd5fac1c2d5,be256df1-75cb-cbf7-0109-e0e1ae2c6348,443402002,Lifestyle education regarding hypertension,59621000.0,Hypertension
2,b70e996d-ad15-b34b-d528-e8150d56379d,2000-01-07,NaN,9bbbcada-7a45-92f0-6ae6-d197bcefc0d4,1b85d074-6a01-7607-27b2-f1e9f6bf305d,698360004,Diabetes self management plan,44054006.0,Diabetes
3,a91f64e4-037e-fff5-6ed0-28c3c0d04ec2,2000-01-07,NaN,9bbbcada-7a45-92f0-6ae6-d197bcefc0d4,a8ea7228-41bd-66d9-12f6-912d0e2dfbe4,443402002,Lifestyle education regarding hypertension,59621000.0,Hypertension
4,cfabbc6f-5086-7399-caa5-1da31e629661,2019-01-30,2019-05-27,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,cd0f83e5-1e70-75ec-1be9-b254e2589e49,53950000,Respiratory therapy,10509002.0,Acute bronchitis (disorder)
...,...,...,...,...,...,...,...,...,...
540,fbe6f1cd-1a0e-5fa2-aba5-10cbf1e3e6e5,1996-04-20,NaN,31698580-0ef9-5510-bea6-0c7ae94b0efc,e8e79c17-3c3e-400f-bf90-579c934e90f1,736285004,Hyperlipidemia clinical management plan,55822004.0,Hyperlipidemia
541,f55749d5-267c-0efb-bba3-eda8b595d054,2014-06-02,2014-08-31,31698580-0ef9-5510-bea6-0c7ae94b0efc,325490ce-c145-7f05-1284-d3fc851a39d0,385691007,Fracture care,33737001.0,Fracture of rib
542,371871a9-c082-2e1b-25b5-ea3dfd3a2e7c,2015-09-25,2015-10-09,31698580-0ef9-5510-bea6-0c7ae94b0efc,57f737c9-e63c-351d-a43e-af2c1da47689,225358003,Wound care,284549007.0,Laceration of hand
543,a346ded2-c434-ff68-cc68-2c567f459d94,2020-02-23,2020-02-23,31698580-0ef9-5510-bea6-0c7ae94b0efc,3690dd8a-8d49-389c-8df9-9ee7c0064f57,736376001,Infectious disease care plan (record artifact),840544004.0,Suspected COVID-19


In [41]:
# insert into table-Larger example- multiple rows
#rows = [(17, '1959-05-31', 'Karl184', 'Stroman', 'M', 'M' , 1500, 'Stroman228@sqlite.net'),
#        (157, '1991-11-27', 'Monty345', 'Krajcik', 'S', 'M' , 400, 'Krajcik437@sqlite.net'),
#        (765, '2001-11-18', 'Hugo693', 'Valencia279', '', 'M' , 566, 'Valencia279@sqlite.net'),
#        (357, '1941-12-22', 'Hayley136', 'Schamberger479', 'S' ,'F', 7000, 'Schamberger479@sqlite.net'),
#        (123, '1951-12-14', 'Jamison785', 'Schoen', 'M', 'M', 9000,'Marvin195@sqlite.net')]
# Preparing SQL queries to INSERT  records into the database.
#cur.executemany('insert into patients_info values(?,?,?,?,?,?,?,?)', rows)
#conn.commit()

zeilenzaehler = 0
while (zeilenzaehler < df.shape[0]):
  cursor.execute("insert into tb_careplans values(?,?,?,?,'allergy',?,'allergy',?,?,?,?,'allergy')", df.iloc[zeilenzaehler,:])
  zeilenzaehler +=1
conn.commit()

In [43]:
cursor.execute("SELECT * FROM tb_careplans")
records = cursor.fetchall()
print("Total rows in table: ", len(records))
for row in records:
  print(row)

Total rows in table:  545
('b066696b-4a2f-ec29-b018-bcf7464d020c', '1943-02-15', None, '3575b903-dbd0-1d55-6146-9e8aa4ed52a5', 'allergy', 'fd59905a-4ec8-62a6-d45d-d2a458d60111', 'allergy', b'\xf1\xf0\xee\x16\x00\x00\x00\x00', 'Self-care interventions (procedure)', None, None, 'allergy')
('8a34101d-6503-4181-6d2d-d316e5f71de0', '2010-01-20', None, 'aff157cc-b6d3-412b-ccbe-bfd5fac1c2d5', 'allergy', 'be256df1-75cb-cbf7-0109-e0e1ae2c6348', 'allergy', b'\x12\xc7m\x1a\x00\x00\x00\x00', 'Lifestyle education regarding hypertension', '59621000.0', 'Hypertension', 'allergy')
('b70e996d-ad15-b34b-d528-e8150d56379d', '2000-01-07', None, '9bbbcada-7a45-92f0-6ae6-d197bcefc0d4', 'allergy', '1b85d074-6a01-7607-27b2-f1e9f6bf305d', 'allergy', b'\xc4 \xa0)\x00\x00\x00\x00', 'Diabetes self management plan', '44054006.0', 'Diabetes', 'allergy')
('a91f64e4-037e-fff5-6ed0-28c3c0d04ec2', '2000-01-07', None, '9bbbcada-7a45-92f0-6ae6-d197bcefc0d4', 'allergy', 'a8ea7228-41bd-66d9-12f6-912d0e2dfbe4', 'allergy', b

In [ ]:
cursor = conn.cursor()
cursor.execute('CREATE TABLE patients (
PATIENT_ID varchar(256) primary key,
#BIRTHDATE date,
#DEATHDATE date
#SSN varchar(256),
#DRIVERS varchar(256),
#PASSPORT varchar(256),
#PREFIX varchar(256),
#FIRST varchar(256),
#LAST varchar(256),
#SUFFIX varchar(256),
#MAIDEN varchar(256),
#MARITAL char(1),
#RACE varchar(256),
#ETHNICITY varchar(256),
#GENDER char(1),
#BIRTHPLACE varchar(256),
#ADDRESS varchar(256),
#CITY varchar(256),
#STATE varchar(256),
#COUNTY varchar(256),
#ZIP varchar(256),
#LAT numeric(8,6),
#LON numeric(8,6),
#HEALTHCARE_EXPENSES numeric(10,2),
#HEALTHCARE_COVERAGE numeric(10,2), 
#DATASET_ORIGIN varchar(256)
  )
')

SyntaxError: ignored

In [ ]:
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO TestDB.dbo.people_info (Name, Country, Age)
                VALUES (?,?,?)
                ''',
                row.Name, 
                row.Country,
                row.Age
                )
conn.commit()